In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import featuretools as ft


In [ ]:
movies = pd.read_csv("./data/raw/movies.csv")
movies.head()

In [ ]:
links = pd.read_csv("./data/raw/links.csv")
links.head()

In [ ]:
ratings = pd.read_csv("./data/raw/ratings.csv")
ratings.head()

In [ ]:
tags = pd.read_csv("./data/raw/tags.csv")
tags.head()

Read in the data from the CSV file

In [18]:
shake = pd.read_csv("./data/raw/Shakespeare_data.csv")
shake.head

<bound method NDFrame.head of         Dataline            Play  PlayerLinenumber ActSceneLine  \
0              1        Henry IV               NaN          NaN   
1              2        Henry IV               NaN          NaN   
2              3        Henry IV               NaN          NaN   
3              4        Henry IV               1.0        1.1.1   
4              5        Henry IV               1.0        1.1.2   
...          ...             ...               ...          ...   
111391    111392  A Winters Tale              38.0      5.3.180   
111392    111393  A Winters Tale              38.0      5.3.181   
111393    111394  A Winters Tale              38.0      5.3.182   
111394    111395  A Winters Tale              38.0      5.3.183   
111395    111396  A Winters Tale              38.0          NaN   

               Player                                         PlayerLine  
0                 NaN                                              ACT I  
1              

Drop the useless Column of data

In [19]:
df_shake = shake.drop(columns="Dataline")
df_shake['Player'].replace(np.nan, 'No Character', inplace = True)

Gets list of unique players

In [20]:
df_PlayerNumber = df_shake.groupby('Player').nunique()

Get the list of times the player talks

In [21]:
val = shake['Player'].value_counts()
val.head

<bound method NDFrame.head of GLOUCESTER          1920
HAMLET              1582
IAGO                1161
FALSTAFF            1117
KING HENRY V        1086
                    ... 
Second Neighbour       1
Second Patrician       1
ARMADO                 1
WORCESTER              1
First Messenger        1
Name: Player, Length: 934, dtype: int64>

In [22]:
df = val.rename_axis('Player').reset_index(name='Number of Appearances')
df.head

<bound method NDFrame.head of                Player  Number of Appearances
0          GLOUCESTER                   1920
1              HAMLET                   1582
2                IAGO                   1161
3            FALSTAFF                   1117
4        KING HENRY V                   1086
..                ...                    ...
929  Second Neighbour                      1
930  Second Patrician                      1
931            ARMADO                      1
932         WORCESTER                      1
933   First Messenger                      1

[934 rows x 2 columns]>

In [23]:
play_grouping = df_shake.groupby(['Play','Player' ]).count()
play_grouping.head

<bound method NDFrame.head of                                      PlayerLinenumber  ActSceneLine  \
Play               Player                                             
A Comedy of Errors ADRIANA                        284           276   
                   AEGEON                         150           147   
                   AEMELIA                         75            73   
                   ANGELO                          99            96   
                   ANTIPHOLUS                       6             6   
...                                               ...           ...   
macbeth            Son                             24            20   
                   Third Apparition                 6             5   
                   Third Murderer                   8             8   
                   Third Witch                     27            27   
                   YOUNG SIWARD                     8             7   

                                     PlayerLin

Imports for both the decision tree and the random forest implmentation

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn import metrics 

So prior to being able to run the classification things, you need to make sure that the processor is able to take in the values. In this case it only takes floats so we need to turn all of our strings into floats by using the label encoder.

In [25]:
le = preprocessing.LabelEncoder()
le.fit(df_shake['Player'])
df_shake['Player'] = le.transform(df_shake['Player'])
le.fit(df_shake['Play'])
df_shake['Play'] = le.transform(df_shake['Play'])

df_shake['ActSceneLine'] = df_shake['ActSceneLine'].astype(str)

le.fit(df_shake['ActSceneLine'])
df_shake['ActSceneLine'] = le.transform(df_shake['ActSceneLine'])

This takes in the X and y classification that we will be using and splitting it into training and testing sets. We will be using Play and ActSceneLine to determine the player.



In [26]:
X= df_shake[['Play', 'ActSceneLine']]
y= df_shake['Player']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Takes in the training and testing set to predict the outcome from our testing set. The accuracy we get returned averages around 60%. The decision tree is quick, but often leads to overfitting of the data.

In [27]:
decision_tree = DecisionTreeClassifier()
clf = decision_tree.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6038599640933573


Like the Decision tree the random forest classifcation takes in the training and testing set to predict the outcome from our testing set. The accuracy we get returned averages around 60%. The random forest classification is again very accurate and fast, but falls into the trap of scaling and the larger the number of trees the slower the classifcation, our dataset begins to show growing pains.

In [12]:
my_model = RandomForestClassifier()
my_model.fit(X_train, y_train)
y_pred = my_model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

c:\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.6043087971274685


All in all we were able to take the shakespeare play and create some feature engineering models to help establish future ways to explain our data. If I had to add one more model, I would have done a way to analyze the playerline in the equation. I think adding that part can lead to an accuracy of over 90%